# Logistic Regression

This notebook is a cheat sheet of logistic regression, which is one of the most commonly used machine learning models for solving binary classification problems. 

## Outline

- [1 - Model Form, Loss Function, Gradient Descent](#1)
- [2 - Logistic Regression in Scikit-learn](#2)
- [3 - Logistic regression with regularization](#3)

<a id='1'></a>
## Model Form, Loss Function, and Gradient Descent for Logistic Regression

### Model Form (Sigmoid Function)

Suppose we have $n$ training examples and $p$ features. Let $X$ and $\vec{y}$ 
denote the feature matrix and target respectively, we can then represent the 
training data as

$$X = 
\left(\begin{array}{cc} 
x_{00}, x_{01}, x_{02}, ..., x_{0p}\\
x_{10}, x_{11}, x_{12}, ..., x_{1p}\\
\vdots\\ 
x_{i0}, x_{i1}, x_{i2}, ..., x_{ip}\\
\vdots\\ 
x_{n0}, x_{n1}, x_{n2}, ..., x_{np}
\end{array}\right),

\vec{y} = 
\left(\begin{array}{cc} 
y_{0}\\
y_{1}\\
\vdots\\ 
y_{i}\\
\vdots\\ 
y_{n}
\end{array}\right)
$$

Let $\vec{x}_i = (x_{i0}, x_{i1}, ..., x_{ip})$, a row vector, denote the $i$-th training example.

Let $\vec{w} = (w_{0}, x_{1}, ..., x_{p})$, a row vector, denote the weights (or parameters) for each feature.

We can then take their dot product to get a linear combination of the features for the $i$-th training example:

$\vec{w} \cdot \vec{x}_i = w_{0}*x_{i0} + w_{1}*x_{i1} + \cdots + w_{p}*x_{ip}$.

Geometrically, this equation is a line/plane/hyperplane passing through the origin (setting all $x$'s values to zero gives zero). To make it not pass the origin, let's add an intercept term $b$ and get a general linear function:

$\vec{w} \cdot \vec{x}_i + b = w_{0}*x_{i0} + w_{1}*x_{i1} + \cdots + w_{p}*x_{ip} + b$.

Let's demonstrate this in code.


In [1]:
import numpy as np

np.random.seed(42)

# create a 5x3 feature matrix of random numbers: 5 examples and 3 features, and
# a target of zeros and ones 
X = np.random.rand(5, 3)          # 2D numpy array
y = np.array([1, 0, 0, 1, 1])     # 1D numpy array
print("Feature matrix shape:", X.shape)
print("Target vector shape:", y.shape)

# initialize the weight vector and the constant
w = np.array([0.1, 0.2, -0.5])    # 1D numpy array
b = 3.5
print("Weight vector shape:", w.shape)

Feature matrix shape: (5, 3)
Target vector shape: (5,)
Weight vector shape: (3,)


In [2]:
# calculate the dot product of the weight vector and the feature vector of the 
# first example (X[0]), because w and X[0] are 1D arrays, we can use the numpy
# function np.dot()
np.dot(w, X[0])

-0.13840009773898304

In [3]:
# add the constant 
np.dot(w, X[0]) + b

3.361599902261017

We are now ready for the model form of logistic regression. Recall that we want to predict a binary target, for example, "yes" or "no", "good" or "bad", "late payment" or "on-time payment", 0 or 1, -1 or 1, and etc. The linear function (of $\vec{w}$ and $b$) above outputs real numbers, and if we can convert them to numbers between 0 and 1, we can interpret them as probabilities and predict positive label if we have a big probability and negative label if we have a small probability. So how do we convert real numbers to numbers between 0 and 1? We use the sigmoid function, $g(z) = \frac{1}{1+e^{-z}}$. If we plug in $z_i = \vec{w} \cdot \vec{x}_i + b$, we get the logistic regression model form $$\hat{y}_i = \frac{1}{1+e^{-(\vec{w} \cdot \vec{x}_i + b)}}$$, where $i=1, ..., n$ is the $i$-th example. 

<center>  <img  src="./images/sigmoid-function.png" width="600" />  

Let's code up the sigmoid function and apply it to the linear function output from the above example.

In [4]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [5]:
sigmoid(np.dot(w, X[0]) + b) # we interpret the output of sigmoid function as probability

0.9664826425608657

### Loss Function (Binary Cross Entropy)


The loss for the $i$-th example (under logistic regression) is

$-y_i*log(\hat{y}_i)-(1-y_i)*log(1-\hat{y}_i)$, where $y_i = 0, 1$ and $\hat{y}_i=\frac{1}{1+e^{-(\vec{w} \cdot \vec{x}_i + b)}}$.

When $y_i$ is 0, the loss simplifies to $-log(1-\hat{y}_i)$. If $\hat{y}_i \approx 0$, the loss will be tiny; if $\hat{y}_i \approx 1$, the loss will blow up.

When $y_i$ is 1, the loss simplifies to $-log(\hat{y}_i)$. If $\hat{y}_i \approx 1$, the loss will be tiny; if $\hat{y}_i \approx 0$, the loss will blow up.

By averaging over losses for all $i = 1, 2, ..., n$ examples, we get the overall loss function (also called binary cross entropy):

$$ L(\vec{w}, b) = \frac{1}{n} \sum_{i=1}^{n} -y_i*log(\hat{y}_i)-(1-y_i)*log(1-\hat{y}_i) $$

Notice the loss is a function of $\vec{w}$ and $b$. It is derived via MLE or MAP. Watch this excellent [lecture](https://www.cs.cornell.edu/courses/cs4780/2018fa/lectures/lecturenote06.html) for details. One thing to note, in the lecture, $\vec{w}$ and $\vec{x}_i$ are column vectors, so $\vec{w}^{T}$ is a row vector.

In [6]:
# let's redo the dot product calculation using column vectors to convince 
# ourself the results are the same regardless how we represent the vectors
w_col_vec = w.reshape(-1, 1)
x0_col_vec = X[0].reshape(-1, 1)
print("w as a column vector has shape", w_col_vec.shape)
print("w transpose as a row vector has shape", w_col_vec.T.shape)
print("x0 as a column vector has shape", x0_col_vec.shape)
print("The dot product of the weights and the 1st example's features values is", np.dot(w_col_vec.T, x0_col_vec)[0,0])

w as a column vector has shape (3, 1)
w transpose as a row vector has shape (1, 3)
x0 as a column vector has shape (3, 1)
The dot product of the weights and the 1st example's features values is -0.13840009773898304


In [7]:
# notice that in the above dot product calculation, we passed a 1x3 matrix (row vector)
# and a 3x1 matrix (column vector) in the function np.dot(). Instead of using np.dot(), 
# we can view it as matrix multiplication and using np.matmul().
print("The dot product of the weights and the 1st example's features values, calculated with `np.matmul()`:", np.matmul(w_col_vec.T, x0_col_vec)[0,0])

The dot product of the weights and the 1st example's features values, calculated with `np.matmul()`: -0.13840009773898304


In [8]:
# The advantange of np.matmul() is that through vectorization, it allows fast 
# computation of dot products between the weight vector and the feature vectors 
# across all examples. This comes handy when you have thousands or millions of examples.
print("The feature matrix has shape", X.shape)
print("The weight vector has shape", w.shape)
Xw = np.matmul(X, w) 
print("The product Xw has shape", Xw.shape)
print("The values of Xw are", Xw)

The feature matrix has shape (5, 3)
The weight vector has shape (3,)
The product Xw has shape (5,)
The values of Xw are [-0.1384001   0.01307232 -0.12151392 -0.41003077  0.0347996 ]


Let's now code up the loss function for logistic regression.

In [9]:
def calc_loss_logistic(X, y, w, b):
    """Computes the loss for logistic regression via vectorization. Fast.

    Args:
        X (ndarray (n,p)): feature matrix
        y (ndarray (n,)) : target values (0 or 1)
        w (ndarray (p,)) : feature weights
        b (scalar)       : intercept
    """
    z = np.matmul(X, w) + b 
    yhat = sigmoid(z)
    loss = -np.sum(y * np.log(yhat) + (1-y) * np.log(1-yhat)) / len(y)
    return loss

In [10]:
calc_loss_logistic(X, y, w, b)

1.412359281839065

### Gradient Descent 

The gradients (partial derivatives) with respect to the model parameters for logistic regression are

$$\frac{\partial{L}}{\partial{w_j}} = \frac{1}{n} \sum_{i=1}^{n} (\hat{y}_i-y_i)x_{ij}$$ 
$$\frac{\partial{L}}{\partial{b}} = \frac{1}{n} \sum_{i=1}^{n} (\hat{y}_i-y_i)$$

where $\hat{y}_i = \frac{1}{1+e^{-(\vec{w} \cdot \vec{x}_i + b)}}$, $i=1, ..., n$ is the $i$-th example, and $j=1, ..., p$ is the $j$-th feature.


Let's code up a function to compute the gradients for logistic regression, given $X$, $\vec{y}$, $\vec{w}$, and $b$.

In [11]:
def calc_gradient_logistic(X, y, w, b):
    """Computes the gradients for logistic regression via vectorization. Fast.

    Args:
        X (ndarray (n,p)): feature matrix
        y (ndarray (n,)) : target values (0 or 1)
        w (ndarray (p,)) : feature weights
        b (scalar)       : intercept
    """
    n = len(y)
    z = np.matmul(X, w) + b
    yhat = sigmoid(z)
    gradient_w = np.matmul(X.T, yhat - y) / n
    gradient_b = np.mean(yhat - y)
    return gradient_w, gradient_b

In [12]:
gradient_w, gradient_b = calc_gradient_logistic(X, y, w, b)
print("Gradient w.r.t. w:", gradient_w)
print("Gradient w.r.t. b:", gradient_b)

Gradient w.r.t. w: [0.11410785 0.19006775 0.13217454]
Gradient w.r.t. b: 0.3665408794153921


In [13]:
def calc_gradient_logistic_slow(X, y, w, b):
    """Computes the loss for logistic regression via for-loops. Slow.

    Args:
        X (ndarray (n,p)): feature matrix
        y (ndarray (n,)) : target values (0 or 1)
        w (ndarray (p,)) : feature weights
        b (scalar)       : intercept
    """    
    m, n = X.shape
    dL_dw = np.zeros(n)
    dL_db = 0
    for i in range(m):
        for j in range(n):
            z_i = np.dot(X[i], w) + b
            yhat_i = sigmoid(z_i)
            dL_dw[j] += (yhat_i - y[i]) * X[i,j]
        dL_db += (yhat_i - y[i])
    dL_dw /= m
    dL_db /= m
    return dL_dw, dL_db

In [14]:
gradient_w, gradient_b = calc_gradient_logistic_slow(X, y, w, b)
print("Gradient w.r.t. w:", gradient_w)
print("Gradient w.r.t. b:", gradient_b)

Gradient w.r.t. w: [0.11410785 0.19006775 0.13217454]
Gradient w.r.t. b: 0.3665408794153921


Now we have a way to calculate the gradients, we can keep iterating and at each iteration, use the **entire** training examples, **simultaneously** update the model parameters according to the following equations:

$$ w_j = w_j - \alpha * \frac{\partial{L}}{\partial{w_j}}, \text{ for } j = 1, ..., p$$

$$ b = b - \alpha * \frac{\partial{L}}{\partial{b}}$$

We can stop when the model parameters stop changing much. This is the batch gradient descent algorithm, where 'batch' means using the entire training dataset at each iteration, and $\alpha$ is the learning rate that we need to choose. If $\alpha$ is tiny, the algorithm will take baby steps and slow to converge. If $\alpha$ is big, the algorithm will take big steps and can easily jump around the local minima and never converge. By graphing the loss over iterations, we can see if we picked a good choice for $\alpha$. If the loss keeps decreasing at reasonable speed, our learning rate works well. If the loss decreases very gradually by little amount over many iterations, our learning rate is too small. If the loss decreases and increases and oscillates, our learning rate is too big. 


<a id='2'></a>
## How to train a logistic regression in Scikit-Learn and make predictions

We'll use a dataset of 2306 [Uniswap V3 LP](https://twitter.com/coindataschool/status/1658997630930944000) positions that I downloaded from [Revert Finance](https://revert.finance/#/top-positions/arbitrum) on 27 Feb 2023. I've filtered and cleaned the data so that it contains equal number of positive (1) and negative (0) labels:

- `is_profitable`: 0 or 1, where 1 means PnL > 0. This is the binary target we want our logistic regression to predict. 

and the following features:
- `rng`: the difference between the upper and lower price limits of the LP position. It's a continuous feature that we can use to train a logistic regression.
- `age`: number of days since position opening. A continuous feature.
- `deposits_value`: dollar value of the deposits at position opening. A continuous feature.

Keep in mind the goal here is NOT to train a model that predicts well, but rather to demonstrate the code for training a logistic regression in scikit-learn. 

In [15]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [16]:
# read univ3 LP metrics on arbitrum
df = pd.read_pickle("../data/univ3_lps_arb.pkl")
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2306 entries, 161248 to 337749
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   is_profitable   2306 non-null   int64  
 1   rng             2306 non-null   float64
 2   age             2306 non-null   float64
 3   deposits_value  2306 non-null   float64
dtypes: float64(3), int64(1)
memory usage: 90.1 KB


In [17]:
# let's separate the target and the features
y = df['is_profitable']
X = df.drop(columns='is_profitable')
print('target:\n', y.value_counts(), '\nshape:', y.shape, '\n')
print('features:\n', X.head(), '\nshape:', X.shape)

target:
 1    1153
0    1153
Name: is_profitable, dtype: int64 
shape: (2306,) 

features:
                 rng        age  deposits_value
nft_id                                        
161248     3.306579  12.414294     3509.477545
120168   509.645056  46.111574      250.430761
342315     4.318640   6.983322      255.209643
326145     2.816710  11.517616     1605.993331
172488  2608.396616  85.732662    11442.193082 
shape: (2306, 3)


Notice each feature is on a different scale. For example, `deposits_value` has large values, whereas `rng`, `age`, and `delta24h_fee_apr` have values that are much smaller. This will cause issues training a logistic regression. The solution is to normalize the features to put them on the same/similar scale. Let's do that now.

In [18]:
# scale the features
scaler = StandardScaler()
Xnorm = scaler.fit_transform(X)
print(f"Peak to Peak range by column in Raw        X:\n{np.ptp(X, axis=0)}\n")
print(f"Peak to Peak range by column in Normalized X:\n{np.ptp(Xnorm, axis=0)}")

Peak to Peak range by column in Raw        X:
rng               3.401887e+50
age               5.383748e+02
deposits_value    7.298510e+06
dtype: float64

Peak to Peak range by column in Normalized X:
[ 6.34735761  7.06871935 17.31835943]


Now that the features are normalized, and the normalized features are more or less on a similar scale, we can train a logistic regression in 2 lines of code using the awesome Scikit-learn package, and use the model to predict the labels of the training examples and calculate accuracy. 

In [19]:
# train model
lr_model = LogisticRegression()
lr_model.fit(Xnorm, y)

# predict on training data
yhat = lr_model.predict(Xnorm)
print("Prediction on training set:", yhat)

# calc training accuracy
print("Accuracy on training set:", lr_model.score(Xnorm, y))

Prediction on training set: [1 1 1 ... 1 1 1]
Accuracy on training set: 0.5082393755420642


A training accuracy of 50.8% is hardly impressive, suggesting our logistic regression is as good as flipping a coin and hence underfit the data. A simple way to address underfitting is to add more features (specifically, non-linear features). It'll give a more complex model, increasing the chance of capturing the relationships between the features and the target. Let's add the 2nd-order terms of the existing features, i.e., squared terms like $x_j^2$ and iteraction terms like $x_jx_k$ and retrain a model.

In [20]:
from sklearn.preprocessing import PolynomialFeatures
# add 2nd order terms 
Xpoly = PolynomialFeatures(degree=2, include_bias=False).fit_transform(X)
print(Xpoly.shape)

(2306, 9)


In [21]:
# scale the features
scaler = StandardScaler()
Xpoly_norm = scaler.fit_transform(Xpoly)
print(f"Peak to Peak range by column in Raw        X:\n{np.ptp(Xpoly, axis=0)}\n")
print(f"Peak to Peak range by column in Normalized X:\n{np.ptp(Xpoly_norm, axis=0)}")

Peak to Peak range by column in Raw        X:
[3.40188746e+050 5.38374768e+002 7.29850977e+006 1.15728383e+101
 1.71194241e+053 4.75080881e+053 2.95251316e+005 3.84247810e+009
 5.32689748e+013]

Peak to Peak range by column in Normalized X:
[ 6.34735761  7.06871935 17.31835943  6.36133299 16.55654053 26.86789077
  8.99955996 18.0801493  19.07802454]


In [22]:
# train model
poly_lr_model = LogisticRegression(max_iter=100000)
poly_lr_model.fit(Xpoly_norm, y)

# predict on training data
yhat = poly_lr_model.predict(Xpoly_norm)
print("Prediction on training set:", yhat)

# calc training accuracy
print("Accuracy on training set:", poly_lr_model.score(Xpoly_norm, y))

Prediction on training set: [0 1 0 ... 0 0 0]
Accuracy on training set: 0.5407632263660017


Adding 2nd-order terms bumps the training accuracy to 0.54. Lol. Let's output the model parameters.

In [23]:
print("w:", poly_lr_model.coef_[0])
print("b:", poly_lr_model.intercept_[0])

w: [ 0.04662739  0.56377792 -0.34371193  0.03398941 -0.40191915  0.12354558
 -0.79012122 -0.26386526  0.58270503]
b: -0.03068733882615702


<a id='3'></a>

## Logistic Regression with Regularization

Adding more features can eventually lead to overfitting, i.e., the model fits too well to the training data, capturing both the signals and noise in the training data, and hence fails to generalize to new data. We can use regularization to reduce overfitting. 

The loss function for (L2) regularized logistic regression is

$$ L(\vec{w}, b) = \frac{1}{n} \sum_{i=1}^{n} -y_i*log(\hat{y}_i)-(1-y_i)*log(1-\hat{y}_i) + \frac{\lambda}{2n} \sum_{j=1}^{p}{w_j^2}$$

and $\sum_{j=1}^{p}{w_j^2}$ is called the L2 regularization term.

The gradients for (L2) regularized logistic regression are

$$\frac{\partial{L}}{\partial{w_j}} = \frac{1}{n} \sum_{i=1}^{n} (\hat{y}_i-y_i)x_{ij} + \frac{\lambda}{n}w_j$$ 
$$\frac{\partial{L}}{\partial{b}} = \frac{1}{n} \sum_{i=1}^{n} (\hat{y}_i-y_i)$$

Remarks:

- L2 (also called Ridge) means squaring the weights in the regularization term. If we take the absolute value instead, we have L1 (also called LASSO) regularization. 
- L2 regularization shrinks the weights toward zero but not equal to zero, whereas L1 regularization sets some weights exactly to zero. 
- L2 regularization is used more often. 
- $\lambda$ is called the regularization strength. The larger the $\lambda$, the smaller the weights.

Let's now train a L2-regularized logistic regression on 9 features (both the 1st-order and 2nd-order terms) of the LP positions data above.  One thing to note, in scikit-learn's logistic regression, there is no $\lambda$ parameter, instead, there's a parameter C, which is $\frac{1}{\lambda}$, so 

- Smaller C: Stronger regularization (more penalty on large coefficients, resulting smaller weights).
- Larger C: Weaker regularization (less penalty on large coefficients, resulting bigger weights).

Let's check this.

In [24]:
# train a L2 regularized logistic regression with C = 10
regu_lr_model = LogisticRegression(C=10, penalty='l2')
regu_lr_model.fit(Xpoly_norm, y)

# check model parameters
print("w:", regu_lr_model.coef_[0])
print("b:", regu_lr_model.intercept_[0])

w: [ 0.05003916  0.58508146 -0.37478974  0.03106631 -0.41784212  0.13111822
 -0.81412158 -0.39189155  0.72538552]
b: -0.03431939971138834


In [25]:
# train a L2 regularized logistic regression with C = 0.01
regu_lr_model = LogisticRegression(C=0.01, penalty='l2')
regu_lr_model.fit(Xpoly_norm, y)

# check model parameters
print("w:", regu_lr_model.coef_[0])
print("b:", regu_lr_model.intercept_[0])

w: [ 0.02623997  0.07879308 -0.06469286  0.02574982 -0.15312429  0.01232702
 -0.22341705 -0.03985615  0.03807066]
b: -0.007199800511723452


We see `C=0.01` results smaller weights (stronger regularization) than `C=10`.

## Summary

In this notebook, we summarized the key knowledge points for logistic regression, including its model form, loss function, gradient function, gradient descent, and L2 regularization. We also demonstrated how to use numpy's dot product and matrix multiplication to implement logistic regression's loss function and gradient calculation via vectorization. Finally, we showed how to train a (regularized) logistic regression in scikit-learn. 

## Learning Resources

- [Supervised Machine Learning: Regression and Classification](https://www.coursera.org/learn/machine-learning).
- [Machine Learning for Intelligent Systems](https://www.cs.cornell.edu/courses/cs4780/2018fa/page18/).

## Referral

- Digital Ocean is a cloud computing platform where you can rent remote servers for cheap. 
  I have my remote data science server there. You can do the same and [get $200 credit](https://m.do.co/c/0a435cb96813). 
